# 🤝 Agent2Agent (A2A) Communication with ADK (Local Version)

This is a modified version of the Kaggle Day 5a notebook, adapted to run in a local VS Code development environment.

## ⚙️ Local Setup

This notebook requires external packages. The following cell will install them into your active Python virtual environment.

In [ ]:
!pip install -q "google-adk[a2a]" python-dotenv

### Configure your Gemini API Key

This notebook uses the [Gemini API](https://ai.google.dev/gemini-api/), which requires an API key.

**1. Create a `.env` file**

In the same folder as this notebook, create a file named `.env`.

**2. Add your key to the `.env` file**

Open the `.env` file and add your API key in the following format:
```
GOOGLE_API_KEY="AIzaSy...your_actual_key_here..."
```

**3. Load the key in the notebook**

Run the cell below to load the key from your `.env` file and set it as an environment variable.

In [1]:
# --- ADD THIS NEW CELL NEAR THE TOP OF YOUR NOTEBOOK ---

import os

# Unset proxy environment variables for this notebook session
# This tells the 'requests' library not to use a proxy.
os.environ['NO_PROXY'] = '127.0.0.1,localhost'
os.environ.pop('HTTP_PROXY', None)
os.environ.pop('HTTPS_PROXY', None)
os.environ.pop('http_proxy', None)
os.environ.pop('https_proxy', None)

print("✅ Proxy environment variables have been cleared for this session.")

✅ Proxy environment variables have been cleared for this session.


In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file in the current directory
load_dotenv()

# Get the API key from the environment variables
# The os.getenv() function looks for a variable named "GOOGLE_API_KEY"
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if GOOGLE_API_KEY:
    print("✅ Setup and authentication complete.")
else:
    print("🔑 Authentication Error: Please make sure you have a .env file with 'GOOGLE_API_KEY' defined.")

✅ Setup and authentication complete.


In [3]:
# --- ADD THIS NEW "SANITY CHECK" CELL ---

import google.generativeai as genai

print("🧪 Performing an API key sanity check...")

try:
    # Configure genai with the key we loaded
    genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
    
    # Create a simple model instance
    model = genai.GenerativeModel('gemini-2.5-flash')
    
    # Try a very simple API call
    response = model.generate_content("hello")
    
    print("✅ Sanity Check PASSED: The API key is valid and working.")
    
except Exception as e:
    print(f"❌ Sanity Check FAILED: The API key is invalid. Please double-check your .env file.")
    print(f"   Error details: {e}")

# --- END OF SANITY CHECK CELL ---

🧪 Performing an API key sanity check...
✅ Sanity Check PASSED: The API key is valid and working.


### Import ADK components

In [4]:
import json
import requests
import subprocess
import time
import uuid

from google.adk.agents import LlmAgent
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)

from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Hide additional warnings in the notebook
import warnings

warnings.filterwarnings("ignore")

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


### Configure Retry Options

In [5]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

## 📦 Section 1: Create the Product Catalog Agent (To Be Exposed)

In [6]:
# Define a product catalog lookup tool
# In a real system, this would query the vendor's product database
def get_product_info(product_name: str) -> str:
    """Get product information for a given product.

    Args:
        product_name: Name of the product (e.g., "iPhone 15 Pro", "MacBook Pro")

    Returns:
        Product information as a string
    """
    # Mock product catalog - in production, this would query a real database
    product_catalog = {
        "iphone 15 pro": "iPhone 15 Pro, $999, Low Stock (8 units), 128GB, Titanium finish",
        "samsung galaxy s24": "Samsung Galaxy S24, $799, In Stock (31 units), 256GB, Phantom Black",
        "dell xps 15": 'Dell XPS 15, $1,299, In Stock (45 units), 15.6" display, 16GB RAM, 512GB SSD',
        "macbook pro 14": 'MacBook Pro 14", $1,999, In Stock (22 units), M3 Pro chip, 18GB RAM, 512GB SSD',
        "sony wh-1000xm5": "Sony WH-1000XM5 Headphones, $399, In Stock (67 units), Noise-canceling, 30hr battery",
        "ipad air": 'iPad Air, $599, In Stock (28 units), 10.9" display, 64GB',
        "lg ultrawide 34": 'LG UltraWide 34" Monitor, $499, Out of Stock, Expected: Next week',
    }

    product_lower = product_name.lower().strip()

    if product_lower in product_catalog:
        return f"Product: {product_catalog[product_lower]}"
    else:
        available = ", ".join([p.title() for p in product_catalog.keys()])
        return f"Sorry, I don't have information for {product_name}. Available products: {available}"


# Create the Product Catalog Agent
product_catalog_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    name="product_catalog_agent",
    description="External vendor's product catalog agent that provides product information and availability.",
    instruction="""
    You are a product catalog specialist from an external vendor.
    When asked about products, use the get_product_info tool to fetch data from the catalog.
    Provide clear, accurate product information including price, availability, and specs.
    If asked about multiple products, look up each one.
    Be professional and helpful.
    """,
    tools=[get_product_info],  # Register the product lookup tool
)

print("✅ Product Catalog Agent created successfully!")

✅ Product Catalog Agent created successfully!


## 🌐 Section 2: Expose the Product Catalog Agent via A2A

In [7]:
product_catalog_a2a_app = to_a2a(
    product_catalog_agent, port=8001  # Port where this agent will be served
)

print("✅ Product Catalog Agent is now A2A-compatible!")

✅ Product Catalog Agent is now A2A-compatible!


## 🚀 Section 3: Start the Product Catalog Agent Server

In [8]:
# --- START OF CORRECTED CELL (SECTION 3) ---

import time
import requests
import subprocess
import os

# First, let's save the product catalog agent to a file that uvicorn can import
product_catalog_agent_code = '''
import os
from dotenv import load_dotenv
from google.adk.agents import LlmAgent
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.genai import types

load_dotenv()
retry_config = types.HttpRetryOptions(attempts=5, exp_base=7, initial_delay=1, http_status_codes=[429, 500, 503, 504])

def get_product_info(product_name: str) -> str:
    """Get product information for a given product."""
    product_catalog = {
        "iphone 15 pro": "iPhone 15 Pro, $999, Low Stock (8 units), 128GB, Titanium finish",
        "samsung galaxy s24": "Samsung Galaxy S24, $799, In Stock (31 units), 256GB, Phantom Black",
        "dell xps 15": 'Dell XPS 15, $1,299, In Stock (45 units), 15.6" display, 16GB RAM, 512GB SSD',
        "macbook pro 14": 'MacBook Pro 14", $1,999, In Stock (22 units), M3 Pro chip, 18GB RAM, 512GB SSD',
        "sony wh-1000xm5": "Sony WH-1000XM5 Headphones, $399, In Stock (67 units), Noise-canceling, 30hr battery",
        "ipad air": 'iPad Air, $599, In Stock (28 units), 10.9" display, 64GB',
        "lg ultrawide 34": 'LG UltraWide 34" Monitor, $499, Out of Stock, Expected: Next week',
    }
    product_lower = product_name.lower().strip()
    if product_lower in product_catalog:
        return f"Product: {product_catalog[product_lower]}"
    else:
        available = ", ".join([p.title() for p in product_catalog.keys()])
        return f"Sorry, I don't have information for {product_name}. Available products: {available}"

product_catalog_agent = LlmAgent(model=Gemini(model="gemini-2.5-flash", retry_options=retry_config), name="product_catalog_agent", description="External vendor's product catalog agent that provides product information and availability.", instruction="""You are a product catalog specialist...""", tools=[get_product_info])
app = to_a2a(product_catalog_agent, port=8001)
'''

with open("product_catalog_server.py", "w") as f:
    f.write(product_catalog_agent_code)

print("📝 Product Catalog agent code saved to product_catalog_server.py")

# --- FIX IS HERE: Using the explicit IP '127.0.0.1' instead of 'localhost' ---
server_process = subprocess.Popen(
    ["uvicorn", "product_catalog_server:app", "--host", "127.0.0.1", "--port", "8001"],
    env=os.environ.copy(),
)

print("🚀 Starting Product Catalog Agent server...")
print("   Waiting for 3 seconds for the server process to initialize...")
time.sleep(3)

print("   Polling server to confirm it's ready...")
max_attempts = 20
for attempt in range(max_attempts):
    try:
        # --- FIX IS HERE: Also polling the explicit IP '127.0.0.1' ---
        response = requests.get("http://127.0.0.1:8001/.well-known/agent-card.json", timeout=1)
        if response.status_code == 200:
            print(f"\\n✅ Product Catalog Agent server is running!")
            break
    except requests.exceptions.RequestException:
        time.sleep(1)
        print(".", end="", flush=True)
else:
    print("\\n⚠️ Server may not be ready. Check the output above for a Python error traceback.")

📝 Product Catalog agent code saved to product_catalog_server.py
🚀 Starting Product Catalog Agent server...
   Waiting for 3 seconds for the server process to initialize...
   Polling server to confirm it's ready...
..

/Users/long-yilee/Documents/VScode/learn_agi_1/product_catalog_server.py:31: UserWarning: [EXPERIMENTAL] to_a2a: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  app = to_a2a(product_catalog_agent, port=8001)
/opt/anaconda3/lib/python3.11/site-packages/google/adk/a2a/utils/agent_to_a2a.py:137: UserWarning: [EXPERIMENTAL] A2aAgentExecutor: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  agent_executor = A2aAgentExecutor(
/opt/anaconda3/lib/python3.11/s

.INFO:     127.0.0.1:58177 - "GET /.well-known/agent-card.json HTTP/1.1" 200 OK
\n✅ Product Catalog Agent server is running!


### 🔍 View the Auto-Generated Agent Card

In [9]:
# Fetch the agent card from the running server
try:
    response = requests.get("http://127.0.0.1:8001/.well-known/agent-card.json", timeout=5)

    if response.status_code == 200:
        agent_card = response.json()
        print("📋 Product Catalog Agent Card:")
        print(json.dumps(agent_card, indent=2))
    else:
        print(f"❌ Failed to fetch agent card: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"❌ Error fetching agent card: {e}")

INFO:     127.0.0.1:58179 - "GET /.well-known/agent-card.json HTTP/1.1" 200 OK
📋 Product Catalog Agent Card:
{
  "capabilities": {},
  "defaultInputModes": [
    "text/plain"
  ],
  "defaultOutputModes": [
    "text/plain"
  ],
  "description": "External vendor's product catalog agent that provides product information and availability.",
  "name": "product_catalog_agent",
  "preferredTransport": "JSONRPC",
  "protocolVersion": "0.3.0",
  "skills": [
    {
      "description": "External vendor's product catalog agent that provides product information and availability. I am a product catalog specialist...",
      "id": "product_catalog_agent",
      "name": "model",
      "tags": [
        "llm"
      ]
    },
    {
      "description": "Get product information for a given product.",
      "id": "product_catalog_agent-get_product_info",
      "name": "get_product_info",
      "tags": [
        "llm",
        "tools"
      ]
    }
  ],
  "supportsAuthenticatedExtendedCard": false,
  "url"

## 🎧 Section 4: Create the Customer Support Agent (Consumer)

In [10]:
# Create a RemoteA2aAgent that connects to our Product Catalog Agent
remote_product_catalog_agent = RemoteA2aAgent(
    name="product_catalog_agent",
    description="Remote product catalog agent from external vendor that provides product information.",
    agent_card=f"http://127.0.0.1:8001{AGENT_CARD_WELL_KNOWN_PATH}",
)

print("✅ Remote Product Catalog Agent proxy created!")

✅ Remote Product Catalog Agent proxy created!


In [11]:
# Now create the Customer Support Agent that uses the remote Product Catalog Agent
customer_support_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    name="customer_support_agent",
    description="A customer support assistant that helps customers with product inquiries and information.",
    instruction="""
    You are a friendly and professional customer support agent.
    When customers ask about products:
    1. Use the product_catalog_agent sub-agent to look up product information
    2. Provide clear answers about pricing, availability, and specifications
    3. If a product is out of stock, mention the expected availability
    Be helpful and professional!
    Always get product information from the product_catalog_agent before answering customer questions.
    """,
    sub_agents=[remote_product_catalog_agent],  # Add the remote agent as a sub-agent!
)

print("✅ Customer Support Agent created!")

✅ Customer Support Agent created!


## 🧪 Section 5: Test A2A Communication

In [12]:
async def test_a2a_communication(user_query: str):
    session_service = InMemorySessionService()
    app_name = "support_app"
    user_id = "demo_user"
    session_id = f"demo_session_{uuid.uuid4().hex[:8]}"

    session = await session_service.create_session(
        app_name=app_name, user_id=user_id, session_id=session_id
    )

    runner = Runner(
        agent=customer_support_agent, app_name=app_name, session_service=session_service
    )
    
    test_content = types.Content(parts=[types.Part(text=user_query)])

    print(f"\n👤 Customer: {user_query}")
    print(f"\n🎧 Support Agent response:")
    print("-" * 60)

    async for event in runner.run_async(
        user_id=user_id, session_id=session_id, new_message=test_content
    ):
        if event.is_final_response() and event.content:
            for part in event.content.parts:
                if hasattr(part, "text"):
                    print(part.text)

    print("-" * 60)

print("🧪 Testing A2A Communication...\n")
await test_a2a_communication("Can you tell me about the iPhone 15 Pro? Is it in stock?")

🧪 Testing A2A Communication...


👤 Customer: Can you tell me about the iPhone 15 Pro? Is it in stock?

🎧 Support Agent response:
------------------------------------------------------------


2025-11-19 17:53:30,643 - INFO - google_llm.py:133 - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-11-19 17:53:34,570 - INFO - google_llm.py:186 - Response received from the model.
2025-11-19 17:53:34,570 - WARNING - types.py:6324 - Warning: there are non-text parts in the response: ['function_call', 'thought_signature'], returning concatenated text result from text parts. Check the full candidates.content.parts accessor to get the full model response.
2025-11-19 17:53:34,591 - INFO - _client.py:1740 - HTTP Request: GET http://127.0.0.1:8001/.well-known/agent-card.json "HTTP/1.1 200 OK"
2025-11-19 17:53:34,592 - INFO - card_resolver.py:83 - Successfully fetched agent card data from http://127.0.0.1:8001/.well-known/agent-card.json: {'capabilities': {}, 'defaultInputModes': ['text/plain'], 'defaultOutputModes': ['text/plain'], 'description': "External vendor's product catalog agent that provides product information and availabilit

INFO:     127.0.0.1:59192 - "GET /.well-known/agent-card.json HTTP/1.1" 200 OK


2025-11-19 17:53:38,837 - INFO - google_llm.py:186 - Response received from the model.
2025-11-19 17:53:38,837 - WARNING - types.py:6324 - Warning: there are non-text parts in the response: ['function_call', 'thought_signature'], returning concatenated text result from text parts. Check the full candidates.content.parts accessor to get the full model response.
/opt/anaconda3/lib/python3.11/site-packages/google/adk/a2a/executor/a2a_agent_executor.py:238: UserWarning: [EXPERIMENTAL] convert_event_to_a2a_events: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  for a2a_event in self._config.event_converter(
/opt/anaconda3/lib/python3.11/site-packages/google/adk/a2a/converters/event_converter.py:556: UserWarning: [EXPERIMEN

INFO:     127.0.0.1:59194 - "POST / HTTP/1.1" 200 OK
The iPhone 15 Pro is available for $999. It has low stock with 8 units remaining. It comes with 128GB storage and a titanium finish.
------------------------------------------------------------


### Try More Examples

In [13]:
await test_a2a_communication(
    "I'm looking for a laptop. Can you compare the Dell XPS 15 and MacBook Pro 14 for me?"
)

2025-11-19 17:53:49,689 - INFO - google_llm.py:133 - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False



👤 Customer: I'm looking for a laptop. Can you compare the Dell XPS 15 and MacBook Pro 14 for me?

🎧 Support Agent response:
------------------------------------------------------------


2025-11-19 17:53:51,372 - INFO - google_llm.py:186 - Response received from the model.
2025-11-19 17:53:51,373 - WARNING - types.py:6324 - Warning: there are non-text parts in the response: ['function_call', 'thought_signature'], returning concatenated text result from text parts. Check the full candidates.content.parts accessor to get the full model response.
2025-11-19 17:53:51,377 - INFO - google_llm.py:133 - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-11-19 17:53:51,379 - INFO - task_manager.py:179 - Task not found or task_id not set. Creating new task for event (task_id: 27877d88-af89-4ff7-96c5-86cbe8f9483c, context_id: dde90301-052a-4af6-b6f5-2f5ebb86d576).
2025-11-19 17:53:52,574 - INFO - google_llm.py:186 - Response received from the model.
2025-11-19 17:53:52,574 - WARNING - types.py:6324 - Warning: there are non-text parts in the response: ['function_call', 'thought_signature', 'function_call'], returning concatenated

INFO:     127.0.0.1:59293 - "POST / HTTP/1.1" 200 OK
The Dell XPS 15 is available for $1,299, has 45 units in stock, features a 15.6" display, 16GB of RAM, and a 512GB SSD.

The MacBook Pro 14" is priced at $1,999, with 22 units in stock. It comes with an M3 Pro chip, 18GB of RAM, and a 512GB SSD.
------------------------------------------------------------


## 🧹 Section 7: Cleanup (Important!)

This final step is crucial for local development. We need to manually stop the background server process that we started earlier. This prevents it from continuing to run after you're done with the notebook.

In [14]:
if 'server_process' in locals() and server_process.poll() is None:
    print("🛑 Stopping the Product Catalog Agent server...")
    server_process.terminate()  # Send a signal to terminate the process
    server_process.wait(timeout=5)  # Wait for it to shut down
    print("✅ Server has been shut down.")
else:
    print("ℹ️ Server was not running or has already been stopped.")

# Clean up the temporary server file
if os.path.exists("product_catalog_server.py"):
    os.remove("product_catalog_server.py")
    print("🗑️ Temporary server file removed.")

🛑 Stopping the Product Catalog Agent server...
✅ Server has been shut down.
🗑️ Temporary server file removed.


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [19594]
